# Introduction:

The purpose of this checkpoint notebook is to apply the methods we used in `04_Characterizing_Demand_Advanced.ipynb` to your cohort by utilizing a measure of your choice. 

In the checkpoint notebooks for `02_Creating_a_cohort.ipynb`, we asked you to create and save your cohort as an SQL table. You will be utilizing the cohort you created as part of this checkpoint notebook. 

> Note: This checkpoint notebook has been created by keeping a TANF exit cohort in mind. We encourage you to reach out to your team facilitator to learn more about the methods you can use to Characterize Demand if your cohort focuses on TANF entry.

At each checkpoint you will be replacing the `___` with the appropriate variable, function or R code snippet. 

Participants are encouraged to attempt the checkpoints on their own. Having said that, hints and suggested solutions are provided and these can be accessed by utilizing the following code:

Hints: `check_#.hint()`

Solutions: `check_#.solution()` – your solutions may vary based on how you define your cohort. We have shared our suggested solutions.

In both cases, # refers to the checkpoint number. For example: we can access the hint and solution for Checkpoint 2 using: `check_2.hint()` and `check_2.solution` respectively. Note: Codes for accessing hints and solutions are currently commented out – in order for these to run, we would need to uncomment them first. 


In [ ]:
options(warn=-1)
# Database interaction imports
suppressMessages(library(odbc))

# For data manipulation/visualization
suppressMessages(library(tidyverse))

# For faster date conversions
suppressMessages(library(lubridate))

# Use percent() function
suppressMessages(library(scales))

# clustering
suppressMessages(library(cluster))
options(warn=0)

# set seed to ensure work is reproducible because k-means has random starting points
set.seed(1)

source('04_Characterizing_Demand_Advanced_checkpoints_hints_solutions.txt')

In [ ]:
# Connect to the server
con <- DBI::dbConnect(odbc::odbc(),
                     Driver = "SQL Server",
                     Server = "msssql01.c7bdq4o2yhxo.us-gov-west-1.rds.amazonaws.com",
                     Trusted_Connection = "True")

In [ ]:
# Code adjusting overall graph attributes
# For easier reading, increase base font size
theme_set(theme_gray(base_size = 16))
# Adjust repr.plot.width and repr.plot.height to change the size of graphs
options(repr.plot.width = 12, repr.plot.height = 8)

## Checkpoint 1: Reading in the year of your interest

For this checkpoint, we ask you to update the code below with the year corresponding to your cohort. For example: If the cohort that you created as part of the checkpoints notebook for `02_Creating_a_cohort.ipynb` was for TANF recipients who exited TANF in '2017 Q2', you will update the code below with year = 2017.

In [ ]:
# read aggregated employer data for a specific year
query <- "select *
from tr_tdc_2022.dbo.employer_yearly_agg
where year = ___"

emp <- dbGetQuery(con, query)

# see employers
head(emp)

In [ ]:
# hint
# check_1.hint()

In [ ]:
# suggested solution
# check_1.solution()

In [ ]:
# do.call constructs and executes a function call from a name or a function and a list of arguments to be passed to it.
# we call the order function on all of emp's columns
emp <- emp[do.call(order, emp), ]

In [ ]:
# Remove features without explanatory power
emp_ml <- emp %>%
    select(-c(Empr_no, year, naics_code, adj_naics_2))

### Examine Scales Across Variables

Use the `str` function to see if there are any categorical variables that remain in your data frame (now called `emp_ml`).

In [ ]:
# Check data type of all variables - make sure all of them are numeric
str(emp_ml)

Convert all variables to a **numeric** type and scale them.

In [ ]:
# convert all numeric variables to numeric type otherwise integer64 won't scale using sapply
emp_ml_num <- emp_ml %>%
    sapply(as.numeric)

In [ ]:
# Scale the features since variables like avg_emp_rate are much smaller than avg_total_earnings
emp_ml_scale <- scale(emp_ml_num)

# View first rows after scaling
emp_ml_scale %>% 
   head()

In [ ]:
dim(emp_ml_scale)

### Analyze Missingness

If an employer has missing information in any of the columns, the row will be dropped in the clustering method.

> Note that you should **never remove data** if possible - in a real world setting you would likely want to fill any missing data with an imputation method or a baseline assumption.

In [ ]:
# Check number of rows (where each row is a unique employer/year combination)
nrow(emp_ml_scale)

In [ ]:
# na.omit will remove any rows with any NA values
emp_ml_scale <- na.omit(emp_ml_scale)

In [ ]:
# Check number of rows after dropping rows with any NA values
nrow(emp_ml_scale)

## Elbow Method

Utilize Elbow Method to choose the appropriate number of clusters, *k*.

In [ ]:
# function to compute total within-cluster sum of square
# we can run this for multiple values of k – showcased later in this notebook
wss <- function(k) {
    kmeans(emp_ml_scale, centers=k, nstart=1)$tot.withinss
}

Utilize `map_dbl` to run the `wss` function for each value of k. 

In [ ]:
# compute and plot wss for k =1 to k = 15
k.values <- 1:15

# extract wss values for each k
wss_values <- map_dbl(k.values, wss)


Once you have `wss_values`, you can plot these using the code below:

In [ ]:
wss_df <- data.frame(wss_values, k.values)

In [ ]:
# plotting wss_df
wss_df %>%
    ggplot(aes(x=k.values, y=wss_values)) + 
    geom_line() + 
    geom_point()

## Checkpoint 2: Choosing *k*
Based on the plot above, choose the appropriate value for *k*. Try choosing a number around the inflection point, where the change in SSE becomes negligible. Store this value by updating the filling in the blank below with your choice of *k*.

In [ ]:
k <- __

In [ ]:
# hint
# check_2.hint()

In [ ]:
# solution
# check_2.solution()

### Try Model

Now that you have chosen **k** using the elbow method, initialize the kmeans model on the scaled employer measures using this value. 

In [ ]:
# Initialize the model and run on emp_ml_scale with centers = 4
k_means <- kmeans(emp_ml_scale, centers = k, nstart = 20)

The output of the `kmeans` function returns the following components:

In [ ]:
names(k_means)

Check the size of each cluster by using the code below:

In [ ]:
# see size of cluster
k_means$size

### Describe Features across Clusters

Select out the `naics_code` and `adj_naics_2` columns.

In [ ]:
emp_few_cols <- emp %>%
    select(-c(naics_code, adj_naics_2))

Remove missing values and create a variable `k.cluster` that allows you to identify the cluster each employer falls in. 

In [ ]:
# remove missing values (none here)
emp_few_cols <- na.omit(emp_few_cols) 

# add cluster number to the original dataframe
frame_4 <- emp_few_cols %>% 
    mutate(k.cluster = k_means$cluster)  


### Summarizing Clusters

Summarize the clusters based on the provided employer measures. 

#### Mean

Use the mean as the summary statistic of interest to summarize your clusters. 

In [ ]:
# remove empr_nbr, year, and naics codes related columns
frame_4_few_cols <- frame_4 %>%
    select(-c(Empr_no, year))

# summarize and add in sizes of each cluster
frame_4_few_cols %>%
    group_by(k.cluster) %>%
    # getting averages for each cluster
    # add suffix "by_employer" to each summarize variable
    summarise(across(everything(), # adds the suffix across every column in our dataframe
                     list(by_employer=mean))) %>%
    mutate(
        size = k_means$size
    ) %>%
    # relocates the size column after the k.cluster columns
    relocate(size, .after=k.cluster)

#### Standard Deviation
Use the standard deviation as the summary statistic of interest to summarize your clusters. 

In [ ]:
# summarize and add in sizes of each cluster
frame_4_few_cols %>%
    group_by(k.cluster) %>%
    # getting averages for each cluster
    # add suffix "by_employer" to each summarize variable
    summarise(across(everything(), # adds the suffix across every column in our dataframe
                     list(by_employer=sd))) %>%
    mutate(
        size = k_means$size
    ) %>%
    # relocates the size column after the k.cluster columns
    relocate(size, .after=k.cluster)

## Checkpoint 3: Linking to cohort 

Update the code below to read in your cohort joined with Indiana's wage records. 



In [ ]:
# read earnings of cohort into R subset to 2017 jobs
qry <- "SELECT nb.*, wr.Empr_no, wr.Wage, wr.yr_quarter
    FROM (tr_tdc_2022.dbo.____ nb
    LEFT JOIN 
    (SELECT SSN, Empr_no, yr_quarter, Wage 
    FROM tr_tdc_2022.dbo.wages_tanf
    WHERE yr_quarter IN ('____', '____', '____', '____')
    ) wr
    ON wr.SSN=nb.SSN)"

df_wages <- dbGetQuery(con, qry)

In [ ]:
# hint
# check_3.hint()

In [ ]:
# solution
# check_3.solution()

Link the `df_wages` dataframe with `emp` to identify what clusters the employees in your cohort fall into. 

In [ ]:
# add cluster number to the original dataframe
frame_4 <- emp %>%                     
    mutate(k.cluster = k_means$cluster)  

# Join wages table with frame_4 clustering results
df_wages_clus <- df_wages %>%
    inner_join(frame_4, by='Empr_no')

#### Number of Employers by cluster

In [ ]:
# see number of employers by cluster that employed someone in the cohort
df_wages_clus %>%
    group_by(k.cluster) %>%
    summarise(emp_cohort = n_distinct(Empr_no))

#### Number of Employers by cluster and quarter

In [ ]:
# see number of employers by cluster that employed someone in the cohort
df_wages_clus %>%
    group_by(k.cluster, yr_quarter) %>%
    summarise(emp_cohort = n_distinct(Empr_no))

#### Comparing within cohort employers to all employers

In [ ]:
# compare within cohort employers to all employers in original clusters
# Get number of unique employers per cluster in the full dataframe (all employers)
frame_4 %>%
    group_by(k.cluster) %>%
    summarise(emp_all = n_distinct(Empr_no))

In [ ]:
# compare with percentages
cohort_emp <- df_wages_clus %>%
    group_by(k.cluster) %>%
    summarise(emp_cohort = n_distinct(Empr_no))

emp_all <- frame_4 %>%
    group_by(k.cluster) %>%
    summarise(emp_all = n_distinct(Empr_no))

# Join cohort employers with all employers, and find percentage
cohort_emp %>%
    inner_join(emp_all, by = 'k.cluster') %>%
    mutate(percentage = (emp_cohort / emp_all) * 100)